# 使用 ESC-50 微调 MiDashengLM

## 运行前检查

在运行之前，请确保 MDL-Toolkit 已正确安装。运行下面的命令，应该输出`mdl-toolkit`的帮助信息。如果命令运行不正确，请检查你的安装。有关更多信息，请参考[安装指南](../docs_zh/installation.md)。

> ### 注意
> 在本地运行时，强烈建议将 MDL-Toolkit 安装到独立的虚拟环境中，以避免依赖项问题。由于 Notebook 中处理虚拟环境较复杂，可以将 MDL-Toolkit 直接安装到 Notebook 所在虚拟环境。

> ### 注意
> 在示例代码执行过程中，将通过网络下载 ESC-50 数据集和 MiDashengLM-7B bf16 精度的完整权重。请确保到 Github 和 Huggingface 的网络连接顺畅，并预留充足存储空间。
>
> 也可以配置 MDL-Toolkit 以从 Modelscope 下载模型。要使用 Modelscope，请确保安装时启用了`modelscope`可选功能，并在`mdl-toolkit`命令中添加`--from-modelscope true`选项。

In [1]:
# 安装 MDL-Toolkit，例如：
# !pip install mdl-toolkit
!mdl-toolkit --help

usage: mdl-toolkit [-h] {train,convert-dataset,inference} ...

options:
  -h, --help            show this help message and exit

subcommands:
  {train,convert-dataset,inference}
    train
    convert-dataset
    inference


## 数据准备

### 下载 ESC-50 数据集并解压缩

运行下面的命令将下载数据集并解压缩。你也可以通过其他方式获取数据集文件，此时后续步骤中的路径需要相应调整。

> ### 网络访问
> 运行该命令将从 Github 下载数据集文件（约 615MiB），可能需要一些时间。请确保网络状况良好，存储空间充足，并耐心等待。

In [2]:
!wget -nc https://github.com/karoldvl/ESC-50/archive/master.zip -O ESC-50.zip
!unzip -o -q ESC-50.zip

File ‘ESC-50.zip’ already there; not retrieving.


现在，ESC-50 数据集应该在`ESC-50-master`目录中可用。

### 将数据集转换为训练所需的格式

ESC-50 提供了 CSV 格式的样本列表，其中样本被分为`1`到`5`总共五个`fold`。数据集的前 10 个样本如下：

In [3]:
!head -n 11 ESC-50-master/meta/esc50.csv

filename,fold,target,category,esc10,src_file,take
1-100032-A-0.wav,1,0,dog,True,100032,A
1-100038-A-14.wav,1,14,chirping_birds,False,100038,A
1-100210-A-36.wav,1,36,vacuum_cleaner,False,100210,A
1-100210-B-36.wav,1,36,vacuum_cleaner,False,100210,B
1-101296-A-19.wav,1,19,thunderstorm,False,101296,A
1-101296-B-19.wav,1,19,thunderstorm,False,101296,B
1-101336-A-30.wav,1,30,door_wood_knock,False,101336,A
1-101404-A-34.wav,1,34,can_opening,False,101404,A
1-103298-A-9.wav,1,9,crow,False,103298,A
1-103995-A-30.wav,1,30,door_wood_knock,False,103995,A


可以使用下面的代码根据`fold`划分训练集和测试集，并格式化数据集以训练模型以格式`category: <category>, target: <target>`预测音频类别。你可以试着调整下面的代码，例如，试着让模型输出 JSON。

In [4]:
import csv
import os
from pathlib import Path

esc50_base = Path("ESC-50-master")
meta_file = esc50_base / "meta" / "esc50.csv"
train_output = Path("train.csv")
test_output = Path("test.csv")

with (
    open(meta_file, "r") as meta,
    open(train_output, "w") as train,
    open(test_output, "w") as test,
):
    reader = csv.DictReader(meta)
    train_writer = csv.DictWriter(train, fieldnames=["audio", "prediction"])
    test_writer = csv.DictWriter(test, fieldnames=["audio", "prediction"])
    train_writer.writeheader()
    test_writer.writeheader()

    for row in reader:
        writer = train_writer if row["fold"] != "5" else test_writer
        writer.writerow(
            {
                "audio": os.fspath(esc50_base / "audio" / row["filename"]),
                "prediction": f"category: {row['category']}, target: {row['target']}",
            }
        )

In [5]:
!echo '==== Train split ===='
!head -n 11 train.csv
!echo '==== Test split  ===='
!head -n 11 test.csv

==== Train split ====
audio,prediction
ESC-50-master/audio/1-100032-A-0.wav,"category: dog, target: 0"
ESC-50-master/audio/1-100038-A-14.wav,"category: chirping_birds, target: 14"
ESC-50-master/audio/1-100210-A-36.wav,"category: vacuum_cleaner, target: 36"
ESC-50-master/audio/1-100210-B-36.wav,"category: vacuum_cleaner, target: 36"
ESC-50-master/audio/1-101296-A-19.wav,"category: thunderstorm, target: 19"
ESC-50-master/audio/1-101296-B-19.wav,"category: thunderstorm, target: 19"
ESC-50-master/audio/1-101336-A-30.wav,"category: door_wood_knock, target: 30"
ESC-50-master/audio/1-101404-A-34.wav,"category: can_opening, target: 34"
ESC-50-master/audio/1-103298-A-9.wav,"category: crow, target: 9"
ESC-50-master/audio/1-103995-A-30.wav,"category: door_wood_knock, target: 30"
==== Test split  ====
audio,prediction
ESC-50-master/audio/5-103415-A-2.wav,"category: pig, target: 2"
ESC-50-master/audio/5-103416-A-2.wav,"category: pig, target: 2"
ESC-50-master/audio/5-103418-A-2.wav,"category: pig, t

### 检查预训练模型效果

MiDashengLM 并不了解 ESC-50 的类别信息，因此直接用于推理时会输出不正确的类别。在本教程中，我们将使用微调方式调整模型的输出，使其与预期相符。在实践中，通过精心设计提示词或添加解码约束，可能无需微调亦可改进模型的输出，你可以根据实际情况选择合适的方式。

MDL-Toolkit 提供了一个便捷推理命令，可以快速运行推理任务，而无需手动编写推理代码。推理命令的输入与训练时使用的格式相同，但无需包含`prediction`列，推理命令会将推理结果放入`prediction`列，并保留所有其他列的内容。由于推理输入格式与训练输入兼容，可以直接使用上面生成的`test.csv`文件进行推理。我们可以使用推理命令观察未微调模型的输出。

参数说明：
* `--model-name mispeech/midashenglm-7b-bf16`：要使用的模型的 Huggingface 名称或本地路径。

> ### 注意
> 本教程使用 bf16 精度模型权重以减少网络和磁盘占用。如果已经拥有 fp32 精度的完整权重，则可以使用`--model-name mispeech/midashenglm-7b`替换`--model-name mispeech/midashenglm-7b-bf16`，以避免重复下载和存储模型权重。

In [6]:
!mdl-toolkit inference \
    test.csv \
    --system-prompt "Output the predicted category in the format of category: <category>, category_id: <category_id>." \
    --output orig-output.csv \
    --model-name mispeech/midashenglm-7b-bf16
! head -n 11 orig-output.csv

Unrecognized keys in `rope_scaling` for 'rope_type'='default': {'mrope_section'}
Loading checkpoint shards: 100%|██████████████████| 4/4 [00:04<00:00,  1.01s/it]
Generating train split: 400 examples [00:00, 7764.21 examples/s]
Processing dataset: 100%|██████████████| 400/400 [00:16<00:00, 23.68 examples/s]
Batching examples (num_proc=32): 100%|█| 400/400 [00:02<00:00, 146.58 examples/s
Inference: 100%|████████████████████████████████| 32/32 [00:33<00:00,  1.05s/it]
audio,prediction
ESC-50-master/audio/5-103415-A-2.wav,"category: livestock, category_id: 1"
ESC-50-master/audio/5-103416-A-2.wav,"category: music, category_id: 1"
ESC-50-master/audio/5-103418-A-2.wav,"category: pig, category_id: 1"
ESC-50-master/audio/5-103420-A-2.wav,"category: animal, category_id: 1"
ESC-50-master/audio/5-103421-A-2.wav,"category: pig, category_id: 1"
ESC-50-master/audio/5-103422-A-2.wav,"category: animal, category_id: 1"
ESC-50-master/audio/5-117118-A-42.wav,"category: alarm, category_id: 1"
ESC-50-master

### 对数据进行转换

为了提高训练效率，我们对数据集进行转换以生成用于训练的数据集格式。下面的命令会对 CSV 文件进行转换，并使用一个简单的系统提示词。你也可以跳过该步骤并在训练时进行转换，此时后续步骤中的数据集路径需要替换为对应的 CSV 文件路径，并且每次训练前都需要一定时间进行转换。

> ### 网络访问
> 运行该命令将从 Huggingface 下载模型分词器。请确保网络状况良好并耐心等待。
>
> 要从 Modelscope 下载模型，请在命令后添加`--from-modelscope true`选项，并确保安装时启用了`modelscope`可选功能。

参数说明：
* `train.csv`：输入 CSV 文件的路径。
* `--output train-converted/`：输出目录的路径，转换后的数据集将保存在该目录中。将会自动创建该目录并覆盖已经存在的内容。
* `--system-prompt ...`：指定一个简单的系统提示词，用于指导模型的行为。

In [7]:
!mdl-toolkit convert-dataset \
    train.csv \
    --output train-converted/ \
    --system-prompt "Output the predicted category in the format of category: <category>, category_id: <category_id>."
!mdl-toolkit convert-dataset \
    test.csv \
    --output test-converted/ \
    --system-prompt "Output the predicted category in the format of category: <category>, category_id: <category_id>."

Generating train split: 1600 examples [00:00, 29889.39 examples/s]
Processing dataset: 100%|████████████| 1600/1600 [00:46<00:00, 34.63 examples/s]
Deriving labels for training (num_proc=32): 100%|█| 1600/1600 [00:02<00:00, 705.
Saving the dataset (2/2 shards): 100%|█| 1600/1600 [00:02<00:00, 617.50 examples
Processing dataset: 100%|██████████████| 400/400 [00:17<00:00, 23.26 examples/s]
Deriving labels for training (num_proc=32): 100%|█| 400/400 [00:01<00:00, 211.03
Saving the dataset (1/1 shards): 100%|█| 400/400 [00:00<00:00, 1333.25 examples/


## 训练模型

我们希望模型在对音频进行分类的同时，能够遵循我们指定的格式输出结果。由于调整格式较为简单，我们将 LoRA rank 设置为 16。由于样本数较少，我们设定每隔 50 步进行一次评估。由于完整模型权重较大，我们使用`bitsandbytes` 4bit 量化版本以减少网络传输和显存占用。该命令将自动使用检测到的加速器加速训练过程，无需手动干预。

> ### 网络访问
> 运行该命令将从 Huggingface 下载模型权重，可能需要一些时间。请确保网络状况良好，存储空间充足，并耐心等待。
>
> 要从 Modelscope 下载模型，请在命令后添加`--from-modelscope true`选项，并确保安装时启用了`modelscope`可选功能。

> ### 注意
> 建议使用高性能 GPU 进行训练，以加快训练速度，MDL-Toolkit 将自动检测并使用可用的 GPU。默认情况下，训练过程将使用单个 GPU。如果你有多张 GPU，请参考[分布式训练指南](../docs_zh/distributed.md)。不要仅使用 CPU 进行训练，否则训练过程会非常缓慢。
>
> 要使用 bf16 精度运行训练，需要约 18GiB 显存，如果显存不足，可以尝试添加`--quantization 8bit`或`--quantization 4bit`，在加载时使用 bitsandbytes 将模型权重量化为8位或4位。注意，量化可能会降低模型的能力，导致次优的输出结果。

参数说明：
* `--lora-rank 32`：设置 LoRA 的 rank 为 32。对于更复杂的任务，可以尝试增加 rank 以提高模型的表达能力。
* `--eval-steps 100`：每隔 100 步进行一次评估。
* `--train-dataset train-converted/`：指定训练数据集的路径。也可以直接指定 CSV 文件的路径，将在训练前完成转换。
* `--eval-dataset test-converted/`：指定评估数据集的路径。可以直接指定 CSV 文件的路径。如果未指定，将不会进行评估。
* `--output output/`：指定输出目录的路径。训练过程中的检查点和训练结果将保存在该目录中。将会自动创建该目录并覆盖已经存在的内容。

In [8]:
!mdl-toolkit train \
     --lora-rank 32 \
     --eval-steps 100 \
     --train-dataset train-converted/ \
     --eval-dataset test-converted/ \
     --output output/ \
     --model-name mispeech/midashenglm-7b-bf16

Distributed: NO
Unrecognized keys in `rope_scaling` for 'rope_type'='default': {'mrope_section'}
Loading checkpoint shards: 100%|██████████████████| 4/4 [00:03<00:00,  1.04it/s]
Model loaded with torch.bfloat16
trainable params: 68,968,448 || all params: 8,350,708,352 || trainable%: 0.8259
Peak VRAM during loading: 15.684 GiB
  0%|                                                   | 0/200 [00:00<?, ?it/s]`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.
{'loss': 3.8798, 'grad_norm': 3.334406852722168, 'learning_rate': 0.0001, 'epoch': 0.01}
{'loss': 3.7007, 'grad_norm': 3.0189285278320312, 'learning_rate': 9.95e-05, 'epoch': 0.01}
{'loss': 3.1371, 'grad_norm': 2.0998125076293945, 'learning_rate': 9.900000000000001e-05, 'epoch': 0.01}
{'loss': 2.8752, 'grad_norm': 2.0570120811462402, 'learning_rate': 9.850000000000001e-05, 'epoch': 0.02}
{'lo

## 进行推理

训练完成后，我们可以使用训练好的模型进行推理。训练结果默认已经合并了 LoRA 适配器，其推理方式与基础模型相同，仅需指定模型路径，即可使用原始模型的推理代码进行推理：

In [9]:
# 在 Notebook 中设置 TOKENIZERS_PARALLELISM=0 以防止警告干扰输出
# 不要在 Notebook 之外设置，否则可能导致性能降低
import os

os.environ["TOKENIZERS_PARALLELISM"] = "0"

import torch
from transformers import AutoModelForCausalLM, AutoProcessor, AutoTokenizer

model_id = "./output/final/"

model = AutoModelForCausalLM.from_pretrained(model_id, trust_remote_code=True, dtype="auto")
tokenizer = AutoTokenizer.from_pretrained(model_id)
processor = AutoProcessor.from_pretrained(model_id, trust_remote_code=True)

model.eval()

messages = [
    {
        "role": "system",
        "content": [
            {"type": "text", "text": "Output the predicted category in the format of category: <category>, category_id: <category_id>."},
        ],
    },
    {
        "role": "user",
        "content": [
            {"type": "audio", "path": "ESC-50-master/audio/5-103415-A-2.wav"},
        ],
    },
]

with torch.no_grad():
    model_inputs = processor.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        add_special_tokens=True,
        return_dict=True,
    ).to(device=model.device, dtype=model.dtype)
    generation = model.generate(**model_inputs)
    output = tokenizer.batch_decode(generation, skip_special_tokens=True)

print(output)

/home/h30/zhoujiahao5/notebook/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Unrecognized keys in `rope_scaling` for 'rope_type'='default': {'mrope_section'}
Loading checkpoint shards: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:01<00:00,  2.57it/s]


['category: pig, target: 2']


我们也可以使用 MDL-Toolkit 的推理命令获取推理结果：

In [10]:
!mdl-toolkit inference \
    test.csv \
    --system-prompt "You are a helpful audio classifier." \
    --user-prompt "Output the predicted category in the format of category: <category>, category_id: <category_id>." \
    --output finetuned-output.csv \
    --model-name ./output/final/

Unrecognized keys in `rope_scaling` for 'rope_type'='default': {'mrope_section'}
Processing dataset: 100%|██████████████| 400/400 [00:10<00:00, 37.31 examples/s]
Batching examples (num_proc=32): 100%|█| 400/400 [00:02<00:00, 147.34 examples/s
Inference: 100%|████████████████████████████████| 32/32 [00:33<00:00,  1.06s/it]


推理结果将保存在指定的输出 CSV 文件中，输出文件的格式与训练时使用的格式相同。经过微调后，模型应该能够使用训练时指定的格式生成输出：

In [11]:
! head -n 11 orig-output.csv
! head -n 11 finetuned-output.csv











ESC-50-master/audio/5-117250-A-2.wav,"category: animal, category_id: 1"
audio,prediction
ESC-50-master/audio/5-103415-A-2.wav,"category: pig, target: 2"
ESC-50-master/audio/5-103416-A-2.wav,"category: door_wood_creaks, target: 33"
ESC-50-master/audio/5-103418-A-2.wav,"category: pig, target: 2"
ESC-50-master/audio/5-103420-A-2.wav,"category: rooster, target: 1"
ESC-50-master/audio/5-103421-A-2.wav,"category: pig, target: 2"
ESC-50-master/audio/5-103422-A-2.wav,"category: pig, target: 2"
ESC-50-master/audio/5-117118-A-42.wav,"category: siren, target: 42"
ESC-50-master/audio/5-117120-A-42.wav,"category: siren, target: 42"
ESC-50-master/audio/5-117122-A-42.wav,"category: siren, target: 42"
ESC-50-master/audio/5-117250-A-2.wav,"category: snoring, target: 28"


## 如何提高性能

恭喜你完成了模型的第一次微调！但是，在不同任务数据上，本教程提供的超参数可能不能达到你心目中理想的效果。如果你对微调结果不满意，可以尝试以下方法：

1. 提高 LoRA Rank，例如使用`--lora-rank 64`。
2. 调整学习率，例如使用`--lr 5e-5`。最佳学习率受到多方面因素影响，可能需要多次尝试或进行系统性超参数搜索才能确定。
3. 调整可训练目标，例如使用`--train-target encoder--train-target projector --train-target decoder --train-target embed_tokens --train-target lm_head`以训练所有可训练目标。在某些情况下，增加可训练目标，特别是`embed_tokens`和`lm_head`，可以改进训练结果。
4. 提高模型精度。如果使用了量化，则应该尝试在没有量化的情况下运行。如果未使用量化，可以使用`--bf16 false`以将模型加载为 fp32 精度。
5. 增加可用训练数据的数量和质量，这可能会改进模型的性能。但重复使用数据，例如将`--num-epochs`设置为大于1的数值，可能效果有限，甚至对性能有负面影响。